# DLPlan: Description Logics State Features for Planning

## 0. Installation

In [ ]:
!pip install --no-cache-dir dlplan==0.2.12 xdot

In [ ]:
import dlplan

## 1. Expand and Visualize First-order State Spaces


In [ ]:
import requests
domain_data = requests.get("https://raw.githubusercontent.com/rleap-project/dlplan/main/colab/delivery/domain.pddl")
instance_data = requests.get("https://raw.githubusercontent.com/rleap-project/dlplan/main/colab/delivery/instance_1_2.pddl")
with open("/content/domain.pddl", "w") as file:
  file.write(domain_data.text)
with open("/content/instance.pddl", "w") as file:
  file.write(instance_data.text)

# 1. Generate state space of Delivery instance with 1 package p and 2 locations a,b
state_space = dlplan.generate_state_space(
    "/content/domain.pddl", 
    "/content/instance.pddl",
    index=0).state_space

In [ ]:
# 2. View state space
from IPython.display import Image
state_space_dot = state_space.to_dot(1)
with open("graph.dot", "w") as file:
    file.write(state_space_dot)
!dot -Tpng graph.dot > graph.png
Image('graph.png')

## 2. Evaluate Description Logics State Features

In [ ]:
# 1. Parse two features
instance_info = state_space.get_instance_info()
vocabulary_info = instance_info.get_vocabulary_info()
factory = dlplan.SyntacticElementFactory(vocabulary_info)
undelivered_packages = factory.parse_concept(
    "c_and(c_not(c_equal(r_primitive(at,0,1),r_primitive(at_g,0,1))),c_primitive(package,0))")
num_undelivered_packages = factory.parse_numerical(
    "n_count(c_and(c_not(c_equal(r_primitive(at,0,1),r_primitive(at_g,0,1))),c_primitive(package,0)))")

In [ ]:
# 2. Evaluate the features
initial_state = state_space.get_states()[state_space.get_initial_state_index()]
print("Undelivered packages in initial state:", 
      undelivered_packages.evaluate(initial_state))
print("Number of undelivered packages in initial states:", 
      num_undelivered_packages.evaluate(initial_state))

Undelivered packages in initial state: {3}
Number of undelivered packages in initial states: 1


# 3. Generate Description Logics State Features

In [ ]:
# Generate and print feature representations
states = state_space.get_states().values()
feature_reprs = dlplan.generate_features(factory, list(states), feature_limit=400)
print("\n".join(feature_reprs))

In [ ]:
# Concept feature was not pruned
assert undelivered_packages.compute_repr() in feature_reprs
# Numerical feature was pruned
assert num_undelivered_packages.compute_repr() not in feature_reprs